In [2]:
import pandas as pd
import os
import numpy as np

In [15]:
PATH = "./wandb_classification_mlp_tp_0505.csv"
dataset = pd.read_csv(PATH)
# PATH = "./wandb_classification_mlp_ccl_0505.csv"
# dataset = pd.read_csv(PATH)
# dataset = pd.concat([dataset, dataset2])

# del dataset["Created"], dataset["Runtime"], dataset["GradC"], dataset["act_B"], dataset["act_F"], dataset["epochs"], dataset["loss_scale_ssl"]
# del dataset["User"], dataset["Group"], dataset["Job Type"], dataset["Tags"], dataset["Sweep"], dataset["Description"], dataset["GPU Count"], dataset["best_epoch"], dataset["num_chn"], dataset["tmax"], dataset["warmup"], dataset["eta_min"]
# if one of the above colum is in the dataset, delete it

for col_name in dataset.columns:
    if col_name in ["Created", "architecture", "batchsize", "activation_function", "agent", "batch_size", "Notes", "yaml", "Runtime", "GradC", "act_B", "act_F", "epochs", "loss_scale_ssl", "User", "Group", "Job Type", "Tags", "Sweep", "Description", "GPU Count", "best_epoch", "num_chn", "tmax", "warmup", "eta_min"]:
        del dataset[col_name]

print("Size of dataset: ", len(dataset))
dataset = dataset[dataset["State"]=="finished"]
# dataset = dataset[dataset["architecture"]=="cnn_pool"]
# dataset = dataset[dataset["batchsize"]==64]
# dataset = dataset[dataset["fw_bn"]==0]
print("Size of dataset: ", len(dataset))
dataset["test_acc"] = dataset["test accuracy"]
dataset["val_acc"] = dataset["valid accuracy"]

Size of dataset:  238
Size of dataset:  232


In [16]:
dataset

,Name,test accuracy,valid accuracy,State,algorithm,dataset,forward_last_activation,learning_rate,loss_feedback,seed,sparse_ratio,std_backward,stepsize,version,test loss,train accuracy,test_acc,val_acc
0,FashionMNIST_BP,0.1010,0.1010,finished,BP,FashionMNIST,linear,0.40,DTP,2,-1,0.01,0.040,NaN,2.160763e+08,0.10045,0.1010,0.1010
1,MNIST_DTP_linear_DTP,0.9714,0.9714,finished,DTP,MNIST,linear,4.00,DTP,1,-1,0.01,0.040,NaN,1.548270e-01,1.00000,0.9714,0.9714
2,CIFAR10_BP,0.4755,0.4755,finished,BP,CIFAR10,linear,0.01,DTP,2,-1,0.01,0.010,NaN,1.916577e+00,0.97950,0.4755,0.4755
3,CIFAR10_FWDTP-BN_linear-BN,0.4627,0.4627,finished,FWDTP-BN,CIFAR10,linear-BN,0.02,DTP,1,-1,0.01,0.010,3.0,1.648343e+00,0.70300,0.4627,0.4627
4,CIFAR100_DTP_linear_DRL,0.1993,0.1993,finished,DTP,CIFAR100,linear,0.80,DRL,4,-1,0.01,0.020,3.0,3.599723e+00,0.28458,0.1993,0.1993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,CIFAR10_DTP_linear_DTP,0.4243,0.4243,finished,DTP,CIFAR10,linear,4.00,DTP,4,-1,0.01,0.002,3.0,2.133082e+00,0.77796,0.4243,0.4243
228,CIFAR100_BP,0.2046,0.2046,finished,BP,CIFAR100,linear,0.02,DTP,2,-1,0.01,0.010,NaN,4.466446e+00,0.93954,0.2046,0.2046
229,MNIST_BP,0.9806,0.9806,finished,BP,MNIST,linear,0.10,DTP,3,-1,0.01,0.040,NaN,7.849678e-02,1.00000,0.9806,0.9806
230,FashionMNIST_DTP_linear_DTP,0.8818,0.8818,finished,DTP,FashionMNIST,linear,1.00,DTP,2,-1,0.01,0.040,3.0,4.014679e-01,0.95825,0.8818,0.8818


In [37]:
for task in ['MNIST', 'FashionMNIST', "CIFAR10", "CIFAR100"]:
    for algorithm in ["BP", "DTP", "FWDTP-BN"]:
        for loss_feedback in ["DTP", "DRL", "L-DRL"]:
            if algorithm == "BP" and loss_feedback != "DTP":
                continue
            if algorithm == "FWDTP-BN" and loss_feedback != "DTP":
                continue
            
            ds = dataset[dataset["dataset"] == task]
            ds = ds[ds["algorithm"] == algorithm]
            ds = ds[ds["loss_feedback"] == loss_feedback]
            # remove duplicates with subset of seed and learning rate  and step_size and batch_size
            ds = ds.drop_duplicates(subset=["seed", "learning_rate", "stepsize", "val_acc"])

            # if task == "CIFAR10" and method == "CCL":
            #     ds = ds[ds["lr_F"] != 1.2]
            ds_mean = ds.groupby(["learning_rate"]).mean()
            ds_count = ds.groupby(["learning_rate"]).count()
            ds_std = ds.groupby(["learning_rate"]).std()
            ds_mean = ds_mean[ds_count["test_acc"] > 2]
            ds_std = ds_std[ds_count["test_acc"] > 2]

            try:
                row_with_highest_val_acc = ds_mean["val_acc"].idxmax()
            except:
                continue
            if algorithm in["BP", "FWDTP-BN"]:
                print("dataset: ", task, " algorithm: ", algorithm)
            else:
                print("dataset: ", task, " algorithm: ",loss_feedback)
            print(f"mean: {ds_mean.loc[row_with_highest_val_acc]['test_acc']*100:.2f} std: {ds_std.loc[row_with_highest_val_acc]['test_acc']*100:.2f} count: {ds_count.loc[row_with_highest_val_acc]['test_acc']}")

            # if np.random.random(1) < 0.1:
            #     assert 0 == 1

dataset:  MNIST  algorithm:  BP
mean: 98.19 std: 0.10 count: 5
dataset:  MNIST  algorithm:  DTP
mean: 97.27 std: 0.06 count: 9
dataset:  MNIST  algorithm:  DRL
mean: 93.05 std: 0.24 count: 10
dataset:  MNIST  algorithm:  L-DRL
mean: 93.29 std: 0.21 count: 10
dataset:  MNIST  algorithm:  FWDTP-BN
mean: 97.20 std: 0.16 count: 9
dataset:  FashionMNIST  algorithm:  BP
mean: 89.58 std: 0.25 count: 5
dataset:  FashionMNIST  algorithm:  DTP
mean: 88.19 std: 0.15 count: 10
dataset:  FashionMNIST  algorithm:  DRL
mean: 83.40 std: 0.18 count: 10
dataset:  FashionMNIST  algorithm:  L-DRL
mean: 83.60 std: 0.20 count: 9
dataset:  FashionMNIST  algorithm:  FWDTP-BN
mean: 88.07 std: 0.28 count: 10
dataset:  CIFAR10  algorithm:  BP
mean: 50.00 std: 0.24 count: 5
dataset:  CIFAR10  algorithm:  DTP
mean: 41.94 std: 0.33 count: 6
dataset:  CIFAR10  algorithm:  DRL
mean: 42.09 std: 0.27 count: 8
dataset:  CIFAR10  algorithm:  L-DRL
mean: 42.19 std: 0.30 count: 6
dataset:  CIFAR10  algorithm:  FWDTP-BN
mea

/var/folders/q7/s2kfc0s17x901pb1hbky39mh0000gn/T/ipykernel_27483/1365600341.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ds_mean = ds.groupby(["learning_rate"]).mean()
/var/folders/q7/s2kfc0s17x901pb1hbky39mh0000gn/T/ipykernel_27483/1365600341.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ds_std = ds.groupby(["learning_rate"]).std()
/var/folders/q7/s2kfc0s17x901pb1hbky39mh0000gn/T/ipykernel_27483/1365600341.py:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or selec

In [36]:
ds

,Name,test accuracy,valid accuracy,State,algorithm,dataset,forward_last_activation,learning_rate,loss_feedback,seed,sparse_ratio,std_backward,stepsize,version,test loss,train accuracy,test_acc,val_acc
7,FashionMNIST_FWDTP-BN_linear-BN,0.8795,0.8795,finished,FWDTP-BN,FashionMNIST,linear-BN,1.0,DTP,1,-1,0.01,0.004,3.0,0.592410,0.964933,0.8795,0.8795
8,FashionMNIST_FWDTP-BN_linear-BN,0.8757,0.8757,finished,FWDTP-BN,FashionMNIST,linear-BN,1.0,DTP,2,-1,0.01,0.004,3.0,0.613204,0.963367,0.8757,0.8757
32,FashionMNIST_FWDTP-BN_linear-BN,0.8807,0.8807,finished,FWDTP-BN,FashionMNIST,linear-BN,1.0,DTP,5,-1,0.01,0.004,NaN,0.600153,0.961050,0.8807,0.8807
75,FashionMNIST_FWDTP-BN_linear-BN,0.8839,0.8839,finished,FWDTP-BN,FashionMNIST,linear-BN,1.0,DTP,1,-1,0.01,0.004,NaN,0.593353,0.964733,0.8839,0.8839
81,FashionMNIST_FWDTP-BN_linear-BN,0.8779,0.8779,finished,FWDTP-BN,FashionMNIST,linear-BN,1.0,DTP,2,-1,0.01,0.004,NaN,0.601886,0.964400,0.8779,0.8779
100,FashionMNIST_FWDTP-BN_linear-BN,0.8801,0.8801,finished,FWDTP-BN,FashionMNIST,linear-BN,1.0,DTP,4,-1,0.01,0.004,3.0,0.599780,0.959800,0.8801,0.8801
117,FashionMNIST_FWDTP-BN_linear-BN,0.8791,0.8791,finished,FWDTP-BN,FashionMNIST,linear-BN,1.0,DTP,4,-1,0.01,0.004,NaN,0.598316,0.964050,0.8791,0.8791
144,FashionMNIST_FWDTP-BN_linear-BN,0.8824,0.8824,finished,FWDTP-BN,FashionMNIST,linear-BN,1.0,DTP,5,-1,0.01,0.004,3.0,0.589759,0.963983,0.8824,0.8824
203,FashionMNIST_FWDTP-BN_linear-BN,0.8831,0.8831,finished,FWDTP-BN,FashionMNIST,linear-BN,1.0,DTP,3,-1,0.01,0.004,NaN,0.595300,0.963283,0.8831,0.8831
231,FashionMNIST_FWDTP-BN_linear-BN,0.8848,0.8848,finished,FWDTP-BN,FashionMNIST,linear-BN,1.0,DTP,3,-1,0.01,0.004,3.0,0.581289,0.963800,0.8848,0.8848


In [23]:
ds_mean

,test accuracy,valid accuracy,seed,sparse_ratio,std_backward,stepsize,version,test loss,train accuracy,test_acc,val_acc
learning_rate,,,,,,,,,,,
1.0,0.88189,0.88189,3.0,-1.0,0.01,0.04,3.0,0.398687,0.955777,0.88189,0.88189
